<a href="https://colab.research.google.com/github/TAUforPython/machinelearning/blob/for-example-and-testing/example_LLM_finetune_Gemma_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies
Run the cell below to install all the required dependencies.

In [1]:
!pip install --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Log into Hugging Face Hub


In [2]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_token"] = userdata.get("HF_token")

In [3]:
from huggingface_hub import login

login(os.environ["HF_token"])

### Loading the model from HF Hub

Supervised Fine-Tuning -> PEFT (Parameter-Efficient Fine-Tuning) - это метод файнтюнинга, который позволяет улучшить результаты предварительно обученных языковых моделей



In [4]:
# сперва необходимо залогинится на HF
# зайти на страницу модели Gemma на HF и там запросить доступ
# https://huggingface.co/google/gemma-1.1-2b-it
#model_id = "google/gemma-1.1-2b-it"

#model_id = "google/gemma-2-2b-it"

#model_id = "HuggingFaceTB/SmolLM-1.7B-Instruct"
#model_id = "meta-llama/Llama-3.2-1B"
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

#model_id = "google/gemma-7b"
#https://huggingface.co/google/gemma-7b

device = "cuda"

In [ ]:
## Load model directly
#from transformers import AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("ruslanmv/Medical-Llama3-v2")
#model = AutoModelForCausalLM.from_pretrained("ruslanmv/Medical-Llama3-v2")

In [5]:
# Let's load the tokenizer first
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Let's quantize the model to reduce its weight
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Let's load the final model
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map={"": 0}
)


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Weights & Biases (WandB) is a machine learning development platform that allows users to track and visualize various aspects of their model training process in real-time.

### Trying it out

In [8]:
prompt = "Some favorite word?"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=32)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

Some favorite word? What is it?

What are some of the most popular words in English?

Please provide your answer as follows:

Words that are frequently used and enjoyed by many people


In [ ]:
prompt = "What can you use an LLM for? Answer:"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

What can you use an LLM for? Answer: As a language model, the main purpose of using an LLM is to provide information and assistance in natural language communication. The following are some examples of how an LLM can be used:

1. Language translation: An LLM can translate text from one language to another, making it useful for international communication.
2. Text generation: An LLM can generate text based on input prompts, such as writing articles or speeches.
3. Information retrieval: An LLM can search large amounts of text data to find relevant information, such as finding news articles based on keywords.
4. Sentiment analysis: An LLM can analyze text data to


## Fine-tuning the model with LoRA

This section of the guide focuses on training your Large Language Model (LLM) to generate famous quotations. Here, we will explore the process of fine-tuning your model to enable it to produce outputs similar to renowned authors, philosophers, and leaders.

In [9]:
# Let's try it out before the fine-tuning
text = "Цитата: учиться, учиться и ещё"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Цитата: учиться, учиться и ещё раз учиться. (1992) - 30\n"Учиться,'

In [10]:
# Loading and processing the dataset
from datasets import load_dataset

#data = load_dataset("Abirate/english_quotes")
data = load_dataset("TutorForTAU/example_modified_quotes")

print("Example item:", data["train"][0])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2509 [00:00<?, ? examples/s]

Example item: {'quote': '“Учиться, учиться и ещё раз учиться”', 'author': 'Lenin Vladimir', 'tags': ['be-yourself', 'Lenin', 'honesty', 'inspirational', 'misattributed-Lenin', 'quote-investigator']}


In [11]:
# Let's tokenize the quotes
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Map:   0%|          | 0/2509 [00:00<?, ? examples/s]

In [12]:
from peft import LoraConfig

# Define tuning parameters
lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [13]:
def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}<eos>"
    return [text]

In [14]:
import transformers
from trl import SFTTrainer

# Create Trainer objects that takes care of the process
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:359: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


In [15]:
!pip install wandb --quiet

In [16]:
import wandb

In [17]:
#os.environ["HF_token"] = userdata.get("HF_token")

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pismodenisu (pismodenisu-ai-is-out-there) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:
run = wandb.init(
    project='Fine-tune on Medical Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [19]:
# Let's run the fine-tuning
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,3.229500
2,3.573300
3,3.238500
4,3.627900
5,3.538700
6,4.485000
7,3.911100
8,3.684600
9,3.682900
10,3.487400


TrainOutput(global_step=10, training_loss=3.6458908557891845, metrics={'train_runtime': 12.3629, 'train_samples_per_second': 3.235, 'train_steps_per_second': 0.809, 'total_flos': 2569411855872.0, 'train_loss': 3.6458908557891845, 'epoch': 0.015942606616181746})

In [20]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=896, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=896, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=896, out_features=128, bias=True)
            (lora_dropout): ModuleDict(
              (default): Identity()
 

In [21]:
new_model = "GemmaFinetune"
trainer.model.save_pretrained(new_model)

In [23]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

NameError: name 'PeftModel' is not defined

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  \nmodel


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [25]:
prompt = "Цитата: учиться, учиться и"

inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=12, do_sample=True, pad_token_id=tokenizer.eos_token_id)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

Цитата: учиться, учиться и учиться, но не учить. - Страниц


In [26]:
# Testing the models after fine-tuning
text = "Quote: Imagination is"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is the key to a successful life. What are some examples of how imagination can be used in everyday life


## Push the model to your Hugging Face Hub


Hugging Face allow to you easily store trained models in their hub.

In [ ]:
# Note: The token needs to have "write" permisssion
#       You can chceck it here:
#       https://huggingface.co/settings/tokens
model.push_to_hub("example-my-gemma-finetuned-model")

model-00002-of-00002.safetensors:   0%|          | 0.00/2.64G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TutorForTAU/example-my-gemma-finetuned-model/commit/09594c275911f32869316270736dd4a497813acb', commit_message='Upload GemmaForCausalLM', commit_description='', oid='09594c275911f32869316270736dd4a497813acb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TutorForTAU/example-my-gemma-finetuned-model', endpoint='https://huggingface.co', repo_type='model', repo_id='TutorForTAU/example-my-gemma-finetuned-model'), pr_revision=None, pr_num=None)